<a href="https://colab.research.google.com/github/Pinokcio/Google-Machine-Learning-Bootcamp-2022/blob/main/Paddy_disease_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/Paddy_Disease_Classification/paddy-disease-classification.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/Paddy_Disease_Classification/')"""

In [ ]:
import csv
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.layers as layers
import tensorflow.keras.optimizers as optimizers
import tensorflow.keras.losses as losses
from tensorflow.keras.callbacks import *
tf.random.set_seed(777)

batch = 32
epoch = 30
image_size1 = 224
image_size2 = 224
lr = 1e-5

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=20, #최대 20도까지 이미지를 회전시킨다.
                                   #width_shift_range=0.2, #수직방향으로 최대 20% 움직인다.
                                   #height_shift_range=0.2, #수평방향으로 최대 20% 움직인다.
                                   shear_range=0.2, #이미지를 최대 20% 자른다.
                                   zoom_range=0.2, #이미지를 최대 20% 확대한다.
                                   #horizontal_flip=True, #이미지가 임의로 좌우로 뒤집힌다.
                                   fill_mode = 'nearest' #손실되어있는 픽셀들을 주변값을 이용해 임의로 채운다.
                                   #validation_split=0.2
                                   )

train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/Paddy_Disease_Classification/train_images', 
        target_size=(image_size1, image_size2),  
        batch_size=batch,
        class_mode='categorical')
        #subset = 'training')

"""validation_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/Paddy_Disease_Classification/train_images', 
        target_size=(image_size1, image_size2),  
        batch_size=batch,
        class_mode='categorical',
        subset = 'validation')"""

#check_point = ModelCheckpoint(filepath='./{epoch}-{val_loss:.2f}-{val_accuracy:.2f}.h5', monitor='val_loss', save_best_only = True, verbose=1)
#early_stopping = EarlyStopping(monitor='val_loss', patience=5)

Found 10407 images belonging to 10 classes.


"validation_generator = train_datagen.flow_from_directory(\n        '/content/drive/MyDrive/Paddy_Disease_Classification/train_images', \n        target_size=(image_size1, image_size2),  \n        batch_size=batch,\n        class_mode='categorical',\n        subset = 'validation')"

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Paddy_Disease_Classification')

from swintransformer import SwinTransformer

model_swin_large_224 = tf.keras.Sequential([
  tf.keras.layers.Lambda(lambda data: tf.keras.applications.imagenet_utils.preprocess_input(tf.cast(data, tf.float32), mode="torch"), input_shape=[image_size1, image_size2, 3]),
  SwinTransformer('swin_large_224', include_top=False, pretrained=True),
  tf.keras.layers.Dense(10, activation='softmax')
])
model_swin_large_224.compile(optimizer = optimizers.Adam(learning_rate=lr), 
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

with tf.device('/device:GPU:0'):
  history = model_swin_large_224.fit(
      train_generator,
      epochs = 15)

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Paddy_Disease_Classification')

from swintransformer import SwinTransformer

model_swin_tiny_224 = tf.keras.Sequential([
  tf.keras.layers.Lambda(lambda data: tf.keras.applications.imagenet_utils.preprocess_input(tf.cast(data, tf.float32), mode="torch"), input_shape=[image_size1, image_size2, 3]),
  SwinTransformer('swin_tiny_224', include_top=False, pretrained=True),
  tf.keras.layers.Dense(10, activation='softmax')
])
model_swin_tiny_224.compile(optimizer = optimizers.Adam(learning_rate=lr), 
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

"""with tf.device('/device:GPU:0'):
  history = model_swin_tiny_224.fit(
      train_generator,
      epochs = epoch)"""

"with tf.device('/device:GPU:0'):\n  history = model_swin_tiny_224.fit(\n      train_generator,\n      epochs = epoch)"

In [ ]:
model_swin_large_224.load_weights('/content/drive/MyDrive/Paddy_Disease_Classification/swinL224_30_aug')

In [ ]:
model_swin_tiny_224.load_weights('/content/drive/MyDrive/Paddy_Disease_Classification/swinT_30_aug')

In [ ]:
"""from tensorflow.keras.applications.efficientnet import EfficientNetB4

pre_trained_model = EfficientNetB4(input_shape = (image_size1, image_size2, 3),
                                  include_top = False)
"""

In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB2  

pre_trained_model = EfficientNetB2(input_shape = (260, 260, 3),
                                  include_top = False)

In [ ]:
def output_of_last_layer(pre_trained_model):
  last_desired_layer = pre_trained_model.get_layer('top_activation')
  last_output = last_desired_layer.output

  return last_output

def create_final_model(pre_trained_model, last_output):
  x = layers.GlobalAveragePooling2D()(last_output)
  x = layers.Dropout(0.2)(x)
  x = layers.Dense(10, activation='softmax')(x)

  model = tf.keras.models.Model(pre_trained_model.input, x)

  model.compile(optimizer = optimizers.Adam(learning_rate=lr), 
                loss = 'categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model


last_output = output_of_last_layer(pre_trained_model)
effieNetB2_t = create_final_model(pre_trained_model, last_output)
effieNetB2_t.load_weights('/content/drive/MyDrive/Paddy_Disease_Classification/efficientNetB2_96_aug')

"""effieNetB2 = create_final_model(pre_trained_model, last_output)
with tf.device('/device:GPU:0'):
  history = effieNetB2.fit(
      train_generator,
      epochs = epoch)"""

"effieNetB2 = create_final_model(pre_trained_model, last_output)\nwith tf.device('/device:GPU:0'):\n  history = effieNetB2.fit(\n      train_generator,\n      epochs = epoch)"

In [ ]:
with tf.device('/device:GPU:0'):
  history = effieNetB2.fit(
      train_generator,
      epochs = epoch,
      initial_epoch=8)

Epoch 9/25
434/434 [==============================] - 227s 521ms/step - loss: 0.5285 - accuracy: 0.8368
Epoch 10/25
434/434 [==============================] - 229s 526ms/step - loss: 0.4837 - accuracy: 0.8515
Epoch 11/25
434/434 [==============================] - 229s 527ms/step - loss: 0.4302 - accuracy: 0.8648
Epoch 12/25
434/434 [==============================] - 229s 526ms/step - loss: 0.3865 - accuracy: 0.8830
Epoch 13/25
434/434 [==============================] - 229s 527ms/step - loss: 0.3449 - accuracy: 0.8952
Epoch 14/25
434/434 [==============================] - 229s 526ms/step - loss: 0.3178 - accuracy: 0.9042
Epoch 15/25
434/434 [==============================] - 229s 527ms/step - loss: 0.2874 - accuracy: 0.9116
Epoch 16/25
434/434 [==============================] - 229s 527ms/step - loss: 0.2601 - accuracy: 0.9198
Epoch 17/25
434/434 [==============================] - 229s 527ms/step - loss: 0.2425 - accuracy: 0.9284
Epoch 18/25
434/434 [==============================] - 2

In [ ]:
with tf.device('/device:GPU:0'):
  history = effieNetB2.fit(
      train_generator,
      epochs = 30,
      initial_epoch=25)

Epoch 26/30
434/434 [==============================] - 227s 521ms/step - loss: 0.1237 - accuracy: 0.9631
Epoch 27/30
434/434 [==============================] - 228s 524ms/step - loss: 0.1179 - accuracy: 0.9650
Epoch 28/30
434/434 [==============================] - 228s 524ms/step - loss: 0.1139 - accuracy: 0.9656
Epoch 29/30
434/434 [==============================] - 228s 525ms/step - loss: 0.1023 - accuracy: 0.9708
Epoch 30/30
434/434 [==============================] - 228s 525ms/step - loss: 0.1004 - accuracy: 0.9697


In [ ]:
with tf.device('/device:GPU:0'):
  history = effieNetB2.fit(
      train_generator,
      epochs = 32,
      initial_epoch=31)

In [ ]:
"""effieNetB2_t = create_final_model(pre_trained_model, last_output)
effieNetB2_t.load_weights('/content/drive/MyDrive/Paddy_Disease_Classification/efficientNetB2_96_aug')"""

In [ ]:
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2L  

pre_trained_model1 = EfficientNetV2L(input_shape = (image_size1, image_size2, 3),
                                  include_top = False)

def output_of_last_layer(pre_trained_model):
  last_desired_layer = pre_trained_model.get_layer('top_activation')
  last_output = last_desired_layer.output

  return last_output

def create_final_model(pre_trained_model, last_output):
  x = layers.GlobalAveragePooling2D()(last_output)
  x = layers.Dropout(0.2)(x)
  x = layers.Dense(10, activation='softmax')(x)

  model = tf.keras.models.Model(pre_trained_model.input, x)

  model.compile(optimizer = optimizers.Adam(learning_rate=lr), 
                loss = 'categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model


last_output = output_of_last_layer(pre_trained_model1)
effieNetV2L_t = create_final_model(pre_trained_model1, last_output)
effieNetV2L_t.load_weights('/content/drive/MyDrive/Paddy_Disease_Classification/efficientNetv2L_97_aug')
"""effieNetV2L = create_final_model(pre_trained_model1, last_output)

with tf.device('/device:GPU:0'):
  history = effieNetV2L.fit(
      train_generator,
      epochs = epoch)"""
  
#effieNetV2L.load_weights('/content/drive/MyDrive/Paddy_Disease_Classification/efficientNetv2L_97_aug')

"effieNetV2L = create_final_model(pre_trained_model1, last_output)\n\nwith tf.device('/device:GPU:0'):\n  history = effieNetV2L.fit(\n      train_generator,\n      epochs = epoch)"

In [ ]:
effieNetV2L = tf.keras.models.load_model('/content/drive/MyDrive/Paddy_Disease_Classification/effieNetV2L.h5')

In [ ]:
"""effieNetV2L.save('/content/drive/MyDrive/Paddy_Disease_Classification/effieNetV2L.h5')
effieNetB2.save('/content/drive/MyDrive/Paddy_Disease_Classification/effieNetB2.h5')"""

In [ ]:
"""model.load_weights('/content/46-0.15-0.96.h5')
model.save_weights('/content/drive/MyDrive/Paddy_Disease_Classification/efficientNetB2_96_aug')"""

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/Paddy_Disease_Classification/test_images', 
        target_size=(image_size1, image_size2),  
        batch_size=batch,
        class_mode='categorical',
        shuffle=False)
#predict = model.predict(test_generator)

Found 3469 images belonging to 1 classes.


In [ ]:
import numpy as np
tta_steps = 5
predictions1 = []

for i in range(tta_steps):
	# generator 초기화
    test_generator.reset()
    
    preds1 = model_swin_large_224.predict_generator(generator = test_generator, verbose = 1)
    predictions1.append(preds1)

# 평균을 통한 final prediction
pred1 = np.mean(predictions1, axis=0)

In [ ]:
predictions2 = []

for i in range(tta_steps):
	# generator 초기화
    test_generator.reset()
    
    preds2 = effieNetB2.predict_generator(generator = test_generator, verbose = 1)
    predictions2.append(preds2)

# 평균을 통한 final prediction
pred2 = np.mean(predictions2, axis=0)

In [ ]:
predictions3 = []

for i in range(tta_steps):
	# generator 초기화
    test_generator.reset()
    
    preds3 = effieNetV2L.predict_generator(generator = test_generator, verbose = 1)
    predictions3.append(preds3)

# 평균을 통한 final prediction
pred3 = np.mean(predictions3, axis=0)

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Paddy_Disease_Classification')

from swintransformer import SwinTransformer

model_swin_tiny_224_t = tf.keras.Sequential([
  tf.keras.layers.Lambda(lambda data: tf.keras.applications.imagenet_utils.preprocess_input(tf.cast(data, tf.float32), mode="torch"), input_shape=[image_size1, image_size2, 3]),
  SwinTransformer('swin_tiny_224', include_top=False, pretrained=True),
  tf.keras.layers.Dense(10, activation='softmax')
])
model_swin_tiny_224_t.compile(optimizer = optimizers.Adam(learning_rate=lr), 
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

with tf.device('/device:GPU:0'):
  history = model_swin_tiny_224_t.fit(
      train_generator,
      epochs = epoch)

Epoch 1/30
326/326 [==============================] - 220s 612ms/step - loss: 2.0686 - accuracy: 0.2485
Epoch 2/30
326/326 [==============================] - 207s 632ms/step - loss: 1.4298 - accuracy: 0.5122
Epoch 3/30
326/326 [==============================] - 207s 634ms/step - loss: 0.9460 - accuracy: 0.6855
Epoch 4/30
326/326 [==============================] - 207s 633ms/step - loss: 0.7172 - accuracy: 0.7625
Epoch 5/30
326/326 [==============================] - 207s 634ms/step - loss: 0.5503 - accuracy: 0.8224
Epoch 6/30
326/326 [==============================] - 207s 634ms/step - loss: 0.4730 - accuracy: 0.8421
Epoch 7/30
326/326 [==============================] - 207s 633ms/step - loss: 0.4000 - accuracy: 0.8681
Epoch 8/30
326/326 [==============================] - 207s 633ms/step - loss: 0.3303 - accuracy: 0.8907
Epoch 9/30
326/326 [==============================] - 207s 634ms/step - loss: 0.2877 - accuracy: 0.9047
Epoch 10/30
326/326 [==============================] - 207s 633m

In [ ]:
pred1 = model_swin_large_224.predict(test_generator)

In [ ]:
"""pred2 = effieNetB2.predict(test_generator) 
pred3 = effieNetV2L.predict(test_generator)"""

In [ ]:
pred_2 = effieNetB2.predict(test_generator)
pred_3 = effieNetV2L.predict(test_generator)

In [ ]:
predict4 = model_swin_tiny_224.predict(test_generator)

In [ ]:
pred5 = effieNetB2_t.predict(test_generator)
pred6 = effieNetV2L_t.predict(test_generator)

In [ ]:
pred7 = model_swin_tiny_224_t.predict(test_generator)

In [ ]:
import pandas as pd
pred_data = pd.read_csv('/content/predict.csv').drop(['Unnamed: 0'],axis=1)

In [ ]:
predict = (pred_data.values*6 + pred7)/7

In [ ]:
argpred = tf.math.argmax(predict, axis=1)

In [ ]:
pd.DataFrame(predict).to_csv('tmp.csv')

In [ ]:
import pandas as pd

data = []
label_dic = {}
for e, i in enumerate(list(train_generator.class_indices)):
  label_dic[e] = i
print(label_dic)
for i in range(200001, 200001+len(argpred)):
  data.append((str(i)+'.jpg', label_dic[int(argpred[i-200001])]))
submission_df = pd.DataFrame(data, columns=['image_id','label'])

submission_df.to_csv("Submission.csv",index=False)

{0: 'bacterial_leaf_blight', 1: 'bacterial_leaf_streak', 2: 'bacterial_panicle_blight', 3: 'blast', 4: 'brown_spot', 5: 'dead_heart', 6: 'downy_mildew', 7: 'hispa', 8: 'normal', 9: 'tungro'}


In [ ]:
"""import matplotlib.pyplot as plt
batch = next(train_generator)
images, labels = batch[0], batch[1] # 0번 이미지데이터 1번 레이블
print(labels[:10])
plt.figure(figsize=(16,8))
for i in range(32) :
    ax = plt.subplot(4,8,i+1)
    plt.imshow(images[i])
    plt.axis("off")
plt.tight_layout()
plt.show()"""